In [7]:
import tensorflow as tf
import numpy as np
import pickle

# local imports
import comm_utils as comm

In [8]:
sps = 16
ofdm_symbol_len = 80  # 64 subcarriers + 16 cyclic prefix
sig_len = 40_960

In [9]:
def get_db(p): return 10*np.log10(p)
def get_pow(s): return np.mean(np.abs(s)**2, axis=-1)
def get_sinr(s, i): return get_pow(s)/get_pow(i)


def get_sinr_db(s, i): return get_db(get_sinr(s, i))


def generate_train_mixture(soi_type, nexamples):
    if soi_type == 'QPSK':
        gen_soi = comm.generate_qpsk_signal
        num_symbols = sig_len // sps
    elif soi_type == 'QPSK_OFDM':
        gen_soi = comm.generate_ofdm_signal
        num_symbols = sig_len // ofdm_symbol_len
    else:
        raise NotImplementedError

    sinr_db = tf.random.uniform(
        shape=(nexamples, 1), minval=-20, maxval=20, dtype=tf.float32)
    gain_linear = tf.pow(10.0, -0.5 * sinr_db / 10.0)
    gain_complex = tf.complex(gain_linear, tf.zeros_like(gain_linear))
    phase = tf.random.uniform(
        shape=(nexamples, 1), minval=-np.pi, maxval=np.pi, dtype=tf.float32)
    phase_complex = tf.complex(phase, tf.zeros_like(gain_linear))
    gain_phasor = gain_complex * tf.math.exp(1j * phase_complex)

    soi, msg_bits = gen_soi(nexamples, num_symbols)
    noise = tf.random.normal(
        shape=(nexamples, sig_len), mean=0.0, stddev=1.0)
    noise_complex = gain_phasor * tf.complex(noise, noise)
    sig_mixed = soi + noise_complex
    return dict(
        soi=soi,
        sig_mixed=sig_mixed.numpy(),
        actual_sinr_db=get_sinr_db(soi, noise_complex),
        expected_sinr_db=sinr_db.numpy(),
        msg_bits=msg_bits.numpy()
    )

**Generate 7000 training examples and 1000 testing examples for training for each signal types**

In [10]:
qpsk_7000 = generate_train_mixture('QPSK', 7000)
qpsk_ofdm_7000 = generate_train_mixture('QPSK_OFDM', 7000)

In [11]:
training_set = dict()
prefix = 'qpsk'
for k in qpsk_7000.keys():
    training_set[(prefix, k)] = qpsk_7000[k]
prefix = 'qpsk_ofdm'
for k in qpsk_ofdm_7000.keys():
    training_set[(prefix, k)] = qpsk_ofdm_7000[k]

training_set.keys()

dict_keys([('qpsk', 'soi'), ('qpsk', 'sig_mixed'), ('qpsk', 'actual_sinr_db'), ('qpsk', 'expected_sinr_db'), ('qpsk', 'msg_bits'), ('qpsk_ofdm', 'soi'), ('qpsk_ofdm', 'sig_mixed'), ('qpsk_ofdm', 'actual_sinr_db'), ('qpsk_ofdm', 'expected_sinr_db'), ('qpsk_ofdm', 'msg_bits')])

In [12]:
with open('./Data/QPSK_7000_QPSK_OFDM_7000.pickle', 'wb') as f:
    pickle.dump(training_set, f)

In [13]:
qpsk_1000 = generate_train_mixture('QPSK', 1000)
qpsk_ofdm_1000 = generate_train_mixture('QPSK_OFDM', 1000)

test_set = dict()
prefix = 'qpsk'
for k in qpsk_1000.keys():
    test_set[(prefix, k)] = qpsk_1000[k]
prefix = 'qpsk_ofdm'
for k in qpsk_ofdm_1000.keys():
    test_set[(prefix, k)] = qpsk_ofdm_1000[k]

test_set.keys()

dict_keys([('qpsk', 'soi'), ('qpsk', 'sig_mixed'), ('qpsk', 'actual_sinr_db'), ('qpsk', 'expected_sinr_db'), ('qpsk', 'msg_bits'), ('qpsk_ofdm', 'soi'), ('qpsk_ofdm', 'sig_mixed'), ('qpsk_ofdm', 'actual_sinr_db'), ('qpsk_ofdm', 'expected_sinr_db'), ('qpsk_ofdm', 'msg_bits')])

In [14]:
with open('./Data/QPSK_1000_QPSK_OFDM_1000.pickle', 'wb') as f:
    pickle.dump(test_set, f)